In [1]:
import os
import settings as s
import dill
import pickle

import torch
import torchtext

import torch.nn as nn

from model import BinarySARNN
from configs import *
from utils import dir_to_csv, tokenize, binary_accuracy, epoch_time

In [2]:
vocab_path = os.path.join(s.DATA_DIR, s.VOCAB_FILE)
with open(vocab_path, "rb") as vf:
    vocab = pickle.load(vf)
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
val_file = os.path.join(s.DATA_DIR, "val.csv")
test_file = os.path.join(s.DATA_DIR, "test.csv")
d_embedding = s.D_EMBEDDING

with open(os.path.join(s.DATA_DIR, "REVIEW.field"), "rb") as reviewf:
    REVIEW = dill.load(reviewf)
LABEL = torchtext.data.Field(sequential=False, use_vocab=False)

val = torchtext.data.TabularDataset(path=val_file, format="CSV",
                                          fields=[("review", REVIEW),
                                                  ("label", LABEL)],
                                          csv_reader_params={"delimiter": " "})

test = torchtext.data.TabularDataset(path=test_file, format="CSV",
                                          fields=[("review", REVIEW),
                                                  ("label", LABEL)],
                                          csv_reader_params={"delimiter": " "})

val_iter = torchtext.data.Iterator(val, 64, device=device, sort_within_batch = True,
                                   sort_key=lambda x: len(x.review))

test_iter = torchtext.data.Iterator(test, 64, device=device, sort_within_batch = True,
                               sort_key=lambda x: len(x.review))



In [23]:
config = LSTM_CONFIG3
config.vocab_size = len(vocab)
config.d_embed = d_embedding

modelc = BinarySARNN(config)
modelc.load_state_dict(torch.load(os.path.join(s.MODELS_PATH, "lstm_2layers_bidir|rmsprop.pt")))
modelc.to(device)

BinarySARNN(
  (embed): Embedding(168994, 300)
  (rnn): LSTM(300, 128, num_layers=2, bidirectional=True)
  (hidden_to_label): Linear(in_features=256, out_features=2, bias=True)
)

In [ ]:
criterion = nn.modules.CrossEntropyLoss()

modelt = modelc
modelt.eval()
tloss, tacc = 0, 0

it = test_iter
with torch.no_grad():
    for test_batch in it:
        treviews, treview_lengths = test_batch.review
        tanswer = modelt(treviews, treview_lengths)
        tacc += binary_accuracy(torch.max(tanswer, 1)[1], test_batch.label).item()
        tloss += criterion(tanswer, test_batch.label)
tloss /= len(it)
tacc /= len(it)

print(f"\tTest Loss: {tloss:.3f} | Test Acc: {tacc*100:.2f}%\n")